# Exogenous Training Notebook (Template + Guide)

해당 노트북은 외생변수 (future exogenous, past exogenous) 를 활용한 데이터셋에서
모델 (PatchTST)를 학습하고 비교하기 위한 템플릿임.

## 작업자(운영자) 체크리스트
1. 아래 **[작업자 입력값]** 섹션에 맞춰 경로/데이터/컬럼명을 채우면됨.
2. 각 시나리오별로 **배치 shape sanity-check** 후 학습을 실행합니다.
3. 결과 모델/로그/플롯 저장 경로를 확인합니다.

> 주의: 데이터 모듈/트레이너 함수(import 경로)는 프로젝트 구조에 맞게 수정이 필요.


In [ ]:
# ------------------------------------------------------------
# Exogenous / Training Config Block
# 목적:
#   - 학습/검증에 필요한 공통 설정(경로, 디바이스, lookback/horizon, 컬럼명, 외생변수 설정)을 정의
#   - 아래 DataModule / Trainer 코드에서 이 변수를 그대로 참조하므로,
#     작업자는 여기에서 "프로젝트/데이터/외생변수"만 교체하면 전체 파이프라인이 동작하도록 구성
# ------------------------------------------------------------

# ------------------------------------------------------------
# (1) Future Exogenous 생성 콜백 유틸 import
# ------------------------------------------------------------
from modeling_module import compose_exo_calendar_cb
# - compose_exo_calendar_cb:
#   date 컬럼을 기반으로 "미래 구간(horizon)"의 캘린더성 외생변수(예: sin/cos 주기) 생성용 콜백을 만든다.
# - trainer/model에서 future_exo_cb가 존재하면,
#   horizon 구간에 대한 외생변수(fe_cont)를 "동적으로 생성"하는 경로로 사용할 수 있다.

import os

import polars as pl
import torch

# ------------------------------------------------------------
# (2) 프로젝트/저장 경로 세팅 (작업자 수정 포인트)
# ------------------------------------------------------------
dir = "C:/Users/USER/PycharmProjects/ts_forecaster_lib/raw_data/"  # TODO(작업자): 본인 로컬/서버 환경에 맞게 프로젝트 raw_data 경로 수정
# - raw_data 디렉토리 아래에 parquet/csv 등 원천 데이터 및 fit(모델 저장) 산출물을 두는 관례
# - Windows 경로이므로 Linux 환경이면 "/home/..." 형태로 변경 필요

save_dir = os.path.join(dir, 'fit')
# - 학습 결과(체크포인트, metrics, 로그 등) 저장하는 상위 폴더
os.makedirs(save_dir, exist_ok=True)

save_root = os.path.join(save_dir, 'Xpatchtst', '20260119')
# - TODO(작업자): 실험명/모델명/날짜(또는 run_id) 기준으로 하위 폴더를 분리해 저장
# - 예: 'Xpatchtst'는 실험 카테고리/모델군, '20260119'는 실험 수행 일자 또는 버전 태그
# - 동일한 save_root를 재사용하면 결과가 덮어써질 수 있으니 실험마다 고유하게 잡는 것을 권장


# ------------------------------------------------------------
# (3) 디바이스 및 학습 기본 파라미터 (작업자 수정 가능)
# ------------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# - GPU 사용 가능하면 cuda, 아니면 cpu
# - 서버/노트북 환경에서 torch가 CUDA 빌드인지 확인하려면 torch.cuda.is_available()가 핵심

lookback = 52
# - 입력 시계열 과거 길이 (주간 기준이면 52주 히스토리)
# - 모델 입력 x shape: [B, lookback, C]가 되는 것이 일반적

horizon = 27
# - 예측 길이 (주간 기준이면 27주 forecast)
# - 모델 출력 y_hat shape: [B, horizon, C] 또는 [B, horizon] 형태

batch_size = 256
# - 배치 크기
# - GPU 메모리/모델 크기/시계열 길이(lookback/horizon)에 따라 OOM 발생 가능 → 필요 시 줄여야 함

freq = 'weekly'
# - 데이터 주기 (weekly/monthly/hourly 등)
# - DateUtil/CalendarExo 생성, DataModule 내부 time index 변환 로직과 강하게 연결됨

split_mode = 'multi'
# - 데이터 split 방식 (프로젝트 내부 정의에 따름)
# - 일반적으로:
#   - 'single': 한 시계열을 train/val로 time-split
#   - 'multi': 다수 시계열(파트/매장 등)을 다루는 multi-series split
# - TODO(작업자): DataModule이 기대하는 enum/문자열과 일치하도록 설정해야 함

shuffle = True
# - DataLoader에서 배치 샘플링을 섞을지 여부
# - 시계열 모델은 "time 순서" 자체는 각 샘플 내부에서 유지되므로,
#   샘플 단위 shuffle은 보통 허용/권장됨
# - 단, 재현성(seed)이나 특정 split 전략에서는 False로 두기도 함


# ------------------------------------------------------------
# (4) 데이터프레임 컬럼명 세팅 (작업자 최우선 수정 포인트)
# ------------------------------------------------------------
id_col = 'unique_id'   # TODO(작업자): 각 시계열을 구분하는 고유 ID 컬럼명
# - 예: part_no, store_id, sku_id 등
# - multi-series 학습에서는 필수

date_col = 'date'      # TODO(작업자): 시점 컬럼명(YYYYWW/날짜/타임스탬프 등)
# - freq='weekly'이면 보통 YYYYWW(int) 또는 실제 date 타입(week start date)로 들어옴
# - compose_exo_calendar_cb가 기대하는 date_type과 일치해야 함

y_col = 'y'            # TODO(작업자): 타깃(수요/판매량) 컬럼명

past_exo_cont_cols = ()  # TODO(작업자): 과거 구간(past, lookback)에 존재하는 연속형 외생변수 컬럼명들
# - 예: price, promo_ratio, past_holiday_flag(연속형으로 넣는다면) 등
# - DataModule이 x와 함께 과거 외생변수를 모델 입력으로 넣는 구조일 때 사용

past_exo_cat_cols = ()   # TODO(작업자): 과거 구간(past)에 존재하는 범주형 외생변수 컬럼명들
# - 예: store_type, region, item_category 등
# - 범주형은 embedding을 태우는 경우가 많으므로 cat_cols로 분리


# ------------------------------------------------------------
# (5) 미래 외생변수(Future Exogenous) 설정: callable 방식
# ------------------------------------------------------------
future_exo_cb = compose_exo_calendar_cb(date_type='W', sincos=True)
# - TODO(작업자): "미래 외생변수"를 어떤 방식으로 공급할지 결정하는 핵심 설정
# - callable 방식의 장점:
#   - 원천 데이터에 미래 구간 외생변수 컬럼이 없어도, date를 기반으로 즉시 생성 가능
#   - 캘린더 기반(주기성) 외생변수(week-of-year sin/cos 등)를 안정적으로 제공
#
# - date_type='W':
#   - Weekly calendar feature를 만든다는 의미
#   - 만약 monthly면 'M', daily면 'D' 등 프로젝트 유틸 정의에 맞춰 변경
#
# - sincos=True:
#   - 주기성을 sin/cos로 인코딩하여 연속형 피처로 반환
#   - 반환되는 fe_cont shape은 [B, horizon, E] 형태 (E는 생성된 외생변수 개수)
#
# - 중요:
#   - loader(데이터셋)에서 이미 fe_cont를 제공하는 경우에도,
#     프로젝트 로직에 따라 future_exo_cb를 우선하거나, loader 값을 우선할 수 있음.
#   - 현재 로그에서 보이듯이:
#       "future_exo_cb=None → skip exo_head setup"
#     같은 분기가 있으므로, 실제 Trainer/Model이 어떤 분기 로직인지 확인해야 함.


# ------------------------------------------------------------
# (6) 실제 데이터 로딩 (작업자 수정 포인트)
# ------------------------------------------------------------
df = pl.read_parquet()
# TODO(작업자): parquet 파일 경로를 반드시 넣어야 함
# 예:
#   df = pl.read_parquet(os.path.join(dir, "walmart_weekly.parquet"))
# 또는:
#   df = pl.read_parquet("C:/data/walmart_weekly.parquet")
#
# - 최소 필요 컬럼:
#   - id_col (unique series id)
#   - date_col (time index)
#   - y_col (target)
#   - + past_exo_cont_cols / past_exo_cat_cols (선택)
#
# - 권장 검증:
#   1) df.columns에 id_col/date_col/y_col 존재 여부 확인
#   2) date_col 타입이 freq와 일치하는지(weekly면 YYYYWW or date)
#   3) 결측치/이상치 처리 여부 (train 전에 사전 클리닝 권장)

In [ ]:
# ------------------------------------------------------------
# Training Execution Block (Weekly) + DataModule / Loader Inspect
# 목적:
#   1) MultiPartExoDataModule로 (x, y, uid, fe, pe_cont, pe_cat) 배치를 구성
#   2) inspect()로 실제 배치 텐서 shape/디바이스/외생변수 주입 상태를 확인
#   3) run_total_train_weekly()로 모델 학습(PatchTST 등)을 실행
# ------------------------------------------------------------

from modeling_module import run_total_train_weekly
# - 전체 학습 파이프라인 엔트리 함수(weekly 전용)
# - 내부적으로:
#   - models_to_run에 지정된 모델을 build/train/validate/save
#   - exogenous_mode, spike 학습, scheduler, early stopping 등이 통합되어 있을 가능성이 큼

from modeling_module import MultiPartExoDataModule
# - MultiPartExoDataModule:
#   - multi-series 데이터를 lookback/horizon 형태로 windowing하여 Dataset/DataLoader 제공
#   - exogenous(과거/미래) 및 positional encoding(pe_cont/pe_cat)까지 배치로 묶어줌


# ------------------------------------------------------------
# (1) Loader sanity check 함수
# ------------------------------------------------------------
def inspect(loader, name):
    """
    DataLoader에서 1 batch를 꺼내 shape과 설정을 확인하는 디버깅 유틸.

    배치 schema (프로젝트 표준):
      x        : 과거 타깃/입력 시계열, shape [B, lookback, C]
      y        : 미래 타깃(정답),       shape [B, horizon,  C] 또는 [B, horizon]
      uid      : 시계열 ID(문자열/정수), shape [B] 또는 [B, 1]
      fe       : 미래 외생변수,         shape [B, horizon, E_future]
      pe_cont  : (연속형) positional/cycle feature 등, shape [B, lookback, E_pe_cont] 등
      pe_cat   : (범주형) positional feature, shape [B, lookback, E_pe_cat] 등

    작업자 체크 포인트:
      - fe.shape[-1] > 0 이면 미래 외생변수가 실제로 배치에 들어오고 있다는 의미
      - loader.collate_fn.future_exo_cb가 None이 아니면,
        collate 시점에 future_exo_cb로 미래 외생을 생성하는 경로가 활성화되었을 가능성이 큼
    """
    b = next(iter(loader))  # 첫 배치를 1회 가져옴
    x, y, uid, fe, pe_cont, pe_cat = b

    # 핵심 입력 텐서 확인
    print(f"[{name}] x:", x.shape, x.device, x.dtype)   # 과거 입력 시계열
    print(f"[{name}] fe:", fe.shape, fe.device, fe.dtype)  # 미래 외생변수
    print(f"[{name}] pe:", pe_cont.shape, pe_cont.device, pe_cont.dtype)  # positional(연속형)

    # 미래 외생 생성 콜백이 붙어있는지 확인
    # - None이면: loader가 미래 외생을 생성하지 않거나, 이미 fe를 df에서 직접 제공하는 방식일 수 있음
    # - None이 아니면: collate 단계에서 date 기반으로 fe를 생성하는 callable 경로 사용 가능성이 큼
    print(f"[{name}] future_exo_cb is None?", loader.collate_fn.future_exo_cb is None)

    # fe가 비어있지 않으면 예시값 출력 (첫 샘플의 앞 3 step, 모든 exo dim)
    # - sin/cos 캘린더면 [-1, 1] 범위 내 값이 주로 관측됨
    if fe.shape[-1] > 0:
        print(f"[{name}] fe sample:", fe[0, :3, :])


# ------------------------------------------------------------
# (2) DataModule 생성: df -> train/val DataLoader 구성
# ------------------------------------------------------------
data_module = MultiPartExoDataModule(
    df=df,                       # (필수) 원본 데이터프레임 (id/date/y 포함)
    id_col=id_col,               # (필수) 시계열 구분 ID 컬럼명
    date_col=date_col,           # (필수) 시점 컬럼명 (weekly면 YYYYWW 또는 date)
    y_col=y_col,                 # (필수) 타깃 컬럼명
    lookback=lookback,           # (필수) 과거 윈도우 길이
    horizon=horizon,             # (필수) 미래 예측 길이
    batch_size=batch_size,       # (필수) 배치 크기

    past_exo_cont_cols=past_exo_cont_cols,  # (선택) 과거 연속형 외생변수 컬럼들
    past_exo_cat_cols=past_exo_cat_cols,    # (선택) 과거 범주형 외생변수 컬럼들

    future_exo_cb=future_exo_cb, # (핵심) 미래 외생 생성 콜백 (캘린더 기반 등)
    # - 주의: 프로젝트 구현에 따라 loader가 df의 미래 exo를 우선할 수도 있고,
    #         future_exo_cb로 생성한 값을 우선할 수도 있음.
    #         inspect() 결과로 fe_dim 및 콜백 활성 여부를 반드시 확인할 것.

    freq=freq,                   # (필수) 'weekly' 등 데이터 주기
    shuffle=shuffle,             # (선택) DataLoader shuffle 여부
    split_mode=split_mode,       # (필수) split 전략 (multi/single 등 프로젝트 정의)
)

# DataLoader 획득
train_loader = data_module.get_train_loader()
val_loader = data_module.get_val_loader()

# ------------------------------------------------------------
# (3) 학습 전 배치 구조 확인 (권장)
# ------------------------------------------------------------
inspect(train_loader, 'train_loader')
# - 작업자 의도:
#   - lookback=52, horizon=27이 맞는지
#   - fe_dim(E_future)이 0인지/양수인지 (외생이 들어오는지)
#   - pe_cont가 기대 shape인지
#   - future_exo_cb가 실제로 collate_fn에 세팅되어 있는지


# ------------------------------------------------------------
# (4) 모델 학습 실행 (Weekly)
# ------------------------------------------------------------
run_total_train_weekly(
    train_loader,
    val_loader,
    device=device,               # 'cuda' 또는 'cpu'
    lookback=lookback,           # train config로 재전달 (로그/모델 빌드에서 사용 가능)
    horizon=horizon,             # train config로 재전달
    warmup_epochs=30,            # (Stage 1) spike OFF 상태로 안정적으로 fitting
    spike_epochs=0,              # (Stage 2) spike ON 학습 추가 epochs (0이면 stage2 미사용)
    save_dir=save_root,          # 모델/로그 저장 경로
    use_exogenous_mode=True,     # (핵심) 모델이 외생변수 입력(fe_cont 등)을 사용하도록 하는 플래그
    # - 주의: 이 값이 True여도, 실제로 fe_dim이 0이면 모델은 외생을 쓰지 못함
    # - 따라서 inspect에서 fe_dim > 0인지 확인이 필수

    models_to_run=['patchtst'],  # 실행할 모델 목록 (예: ['patchtst','titan', 'patchmixer])
    use_ssl_pretrain=True        # PatchTST self-supervised pretrain 경로 사용 여부 (프로젝트 구현에 의존)
)

# 작업자 체크 포인트 (실행 후 로그에서 확인)
#   1) "[total_train] future exo from loader: fe_dim=..." 이 기대값인지
#   2) "[EXO-setup] ... has_head=True/False" 등 exo_head 구성 여부가 의도대로인지
#   3) Train/Val loss가 비정상적으로 튀면:
#        - fe scale/정규화(특히 미래 외생)
#        - spike loss 사용 여부(spike_epochs)
#        - target(y) 스케일 (RevIN/정규화 적용 여부)
#      를 우선 점검